In [1]:
%tensorflow_version 1.x
!git clone https://github.com/tensorflow/cleverhans.git
!pip install cleverhans/
!pip install adversarial-robustness-toolbox

TensorFlow 1.x selected.
Cloning into 'cleverhans'...
remote: Enumerating objects: 13501, done.
remote: Total 13501 (delta 0), reused 0 (delta 0), pack-reused 13501
Receiving objects: 100% (13501/13501), 8.40 MiB | 24.73 MiB/s, done.
Resolving deltas: 100% (9494/9494), done.
Processing ./cleverhans
     |████████████████████████████████| 163kB 2.9MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
  Created wheel for cleverhans: filename=cleverhans-3.0.1-cp36-none-any.whl size=253453 sha256=6637460de83afed549a1ae82f4b4519eeb453a1d191e75a55b35a365bf958b8d
  Stored in directory: /tmp/pip-ephem-wheel-cache-_1e1wf41/wheels/d1/6b/1d/5cf7b3ca4c0cfc7f845628b8ed46366ab5f4f56b5483e9db7f
Successfully built cleverhans
     |████████████████████████████████| 491kB 2.8MB/s 
     |████████████████████████████████| 7.1MB 41.6MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [37]:
  
from __future__ import absolute_import, division, print_function, unicode_literals
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
import numpy as np
import matplotlib.pyplot as plt
from art.classifiers import KerasClassifier
from art.utils import load_dataset
from art.attacks import FunctionallyEquivalentExtraction
import tensorflow as tf
import tensorflow_hub as hub
tf.compat.v1.disable_eager_execution()

img_rows = 32
img_cols = 32
number_channels = 3

# Step 1: Load the CIFAR 10 dataset
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str("cifar10")) # Original Dataset
print("x_train shape: " + str(x_train.shape) + "\n" + "x_train size: " + str(x_train.size) + "\n" +
      "y_train shape: " + str(y_train.shape) + "\n" + "y_train size: " + str(y_train.size) + "\n" +
      "x_test shape: " + str(x_test.shape) + "\n" + "x_test size: " + str(x_test.size) + "\n" +
      "y_test shape: " + str(y_test.shape) + "\n" + "y_test size: " + str(y_test.size) + "\n")

input_shape = (number_channels * img_rows * img_cols,)
print(input_shape)

x_train = x_train.reshape((x_train.shape[0], number_channels * img_rows * img_cols)).astype("float64")
x_test = x_test.reshape((x_test.shape[0], number_channels * img_rows * img_cols)).astype("float64")

mean = np.mean(x_train)
std = np.std(x_train)

x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

y_train = tf.keras.utils.to_categorical(y_train, number_classes)
y_test = tf.keras.utils.to_categorical(y_test, number_classes)

print("x_train shape: " + str(x_train.shape) + "\n" + "x_train size: " + str(x_train.size) + "\n" +
      "y_train shape: " + str(y_train.shape) + "\n" + "y_train size: " + str(y_train.size) + "\n" +
      "x_test shape: " + str(x_test.shape) + "\n" + "x_test size: " + str(x_test.size) + "\n" +
      "y_test shape: " + str(y_test.shape) + "\n" + "y_test size: " + str(y_test.size) + "\n")


# Step 2: Load the victim model
hub_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
embed = hub.KerasLayer(hub_url, input_shape=(32,32,3) )
model = tf.keras.Sequential([
    embed,
    tf.keras.layers.Activation( 'softmax' )
])
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()
victim_classifier = KerasClassifier(model=model, use_logits=True, clip_values=(0., 1.))

# Step 3: Evaluate the victim model on the benign dataset
predictions = victim_classifier.predict(x_test) # giving the classifier the x_test of the CIFAR-10 dataset.
accuracy_benign = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test) # calculates the accuracy of the predictions
print("Accuracy on benign test examples for victim classifier: {}%\n".format(accuracy_benign * 100))


x_train shape: (50000, 32, 32, 3)
x_train size: 153600000
y_train shape: (50000, 10)
y_train size: 500000
x_test shape: (10000, 32, 32, 3)
x_test size: 30720000
y_test shape: (10000, 10)
y_test size: 100000

(3072,)
x_train shape: (50000, 3072)
x_train size: 153600000
y_train shape: (50000, 10, 10)
y_train size: 5000000
x_test shape: (10000, 3072)
x_test size: 30720000
y_test shape: (10000, 10, 10)
y_test size: 1000000



Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_5 (KerasLayer)   (None, 10)                7796426   
_________________________________________________________________
activation_5 (Activation)    (None, 10)                0         
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________


ValueError: ignored

In [28]:
img_rows = 32
img_cols = 32
number_channels = 3

input_shape = (number_channels * img_rows * img_cols,)
print(input_shape)

x_train = x_train.reshape((x_train.shape[0], number_channels * img_rows * img_cols)).astype("float64")
x_test = x_test.reshape((x_test.shape[0], number_channels * img_rows * img_cols)).astype("float64")

mean = np.mean(x_train)
std = np.std(x_train)

x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

y_train = tf.keras.utils.to_categorical(y_train, number_classes)
y_test = tf.keras.utils.to_categorical(y_test, number_classes)

print("x_train shape: " + str(x_train.shape) + "\n" + "x_train size: " + str(x_train.size) + "\n" +
      "y_train shape: " + str(y_train.shape) + "\n" + "y_train size: " + str(y_train.size) + "\n" +
      "x_test shape: " + str(x_test.shape) + "\n" + "x_test size: " + str(x_test.size) + "\n" +
      "y_test shape: " + str(y_test.shape) + "\n" + "y_test size: " + str(y_test.size) + "\n")


(3072,)
x_train shape: (50000, 3072)
x_train size: 153600000
y_train shape: (50000, 10, 10)
y_train size: 5000000
x_test shape: (10000, 3072)
x_test size: 30720000
y_test shape: (10000, 10, 10)
y_test size: 1000000



In [27]:
print(x_train[0:100].shape)

extraction_attack = FunctionallyEquivalentExtraction(classifier=victim_classifier, num_neurons=3072) 
stolen_classifier = extraction_attack.extract(x_train[0:100])
#stolen_classifier = extraction_attack._o_l(x=x_train)

(100, 32, 32, 3)


ValueError: ignored

In [38]:
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

np.random.seed(0)

number_neurons = 16

batch_size = 128
number_classes = 10
epochs = 10
img_rows = 28
img_cols = 28
number_channels = 1

(x_train, y_train), (x_test, y_test) = mnist.load_data()
print("x_train shape: " + str(x_train.shape) + "\n" + "x_train size: " + str(x_train.size) + "\n" +
      "y_train shape: " + str(y_train.shape) + "\n" + "y_train size: " + str(y_train.size) + "\n" +
      "x_test shape: " + str(x_test.shape) + "\n" + "x_test size: " + str(x_test.size) + "\n" +
      "y_test shape: " + str(y_test.shape) + "\n" + "y_test size: " + str(y_test.size) + "\n")

x_train shape: (60000, 28, 28)
x_train size: 47040000
y_train shape: (60000,)
y_train size: 60000
x_test shape: (10000, 28, 28)
x_test size: 7840000
y_test shape: (10000,)
y_test size: 10000



In [39]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, number_channels)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, number_channels)
print("x_train shape: " + str(x_train.shape) + "\n" + "x_train size: " + str(x_train.size) + "\n" +
      "y_train shape: " + str(y_train.shape) + "\n" + "y_train size: " + str(y_train.size) + "\n" +
      "x_test shape: " + str(x_test.shape) + "\n" + "x_test size: " + str(x_test.size) + "\n" +
      "y_test shape: " + str(y_test.shape) + "\n" + "y_test size: " + str(y_test.size) + "\n")

x_train shape: (60000, 28, 28, 1)
x_train size: 47040000
y_train shape: (60000,)
y_train size: 60000
x_test shape: (10000, 28, 28, 1)
x_test size: 7840000
y_test shape: (10000,)
y_test size: 10000



In [40]:
input_shape = (number_channels * img_rows * img_cols,)
print(input_shape)

x_train = x_train.reshape((x_train.shape[0], number_channels * img_rows * img_cols)).astype("float64")
x_test = x_test.reshape((x_test.shape[0], number_channels * img_rows * img_cols)).astype("float64")

mean = np.mean(x_train)
std = np.std(x_train)

x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

y_train = tf.keras.utils.to_categorical(y_train, number_classes)
y_test = tf.keras.utils.to_categorical(y_test, number_classes)

print("x_train shape: " + str(x_train.shape) + "\n" + "x_train size: " + str(x_train.size) + "\n" +
      "y_train shape: " + str(y_train.shape) + "\n" + "y_train size: " + str(y_train.size) + "\n" +
      "x_test shape: " + str(x_test.shape) + "\n" + "x_test size: " + str(x_test.size) + "\n" +
      "y_test shape: " + str(y_test.shape) + "\n" + "y_test size: " + str(y_test.size) + "\n")

(784,)
x_train shape: (60000, 784)
x_train size: 47040000
y_train shape: (60000, 10)
y_train size: 600000
x_test shape: (10000, 784)
x_test size: 7840000
y_test shape: (10000, 10)
y_test size: 100000



In [41]:
model = Sequential()
model.add(Dense(number_neurons, activation="relu", input_shape=input_shape))
model.add(Dense(number_classes, activation="linear"))

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001,),
    metrics=["accuracy"],
)
print(x_train.shape)
print(y_train.shape)
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

model.save("./model.h5")

score_target = model.evaluate(x_test, y_test, verbose=0)

target_classifier = KerasClassifier(model=model, use_logits=True, clip_values=(0, 1))


(60000, 784)
(60000, 10)
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 29us/sample - loss: 1.5431 - acc: 0.5148 - val_loss: 0.9395 - val_acc: 0.7228
Epoch 2/10
60000/60000 [==============================] - 1s 23us/sample - loss: 0.7034 - acc: 0.8022 - val_loss: 0.5126 - val_acc: 0.8595
Epoch 3/10
60000/60000 [==============================] - 1s 23us/sample - loss: 0.4692 - acc: 0.8690 - val_loss: 0.3961 - val_acc: 0.8902
Epoch 4/10
60000/60000 [==============================] - 1s 23us/sample - loss: 0.3860 - acc: 0.8903 - val_loss: 0.3439 - val_acc: 0.9054
Epoch 5/10
60000/60000 [==============================] - 1s 23us/sample - loss: 0.3419 - acc: 0.9028 - val_loss: 0.3128 - val_acc: 0.9132
Epoch 6/10
60000/60000 [==============================] - 1s 23us/sample - loss: 0.3137 - acc: 0.9108 - val_loss: 0.2926 - val_acc: 0.9174
Epoch 7/10
60000/60000 [==============================] - 1s 22us/sample - loss: 0.2933 - a

In [0]:
print(x_test.shape)
print(number_neurons)
fee = FunctionallyEquivalentExtraction(classifier=target_classifier, num_neurons=number_neurons)
bbc = fee.extract(x_test[0:100])


(10000, 784)
16


/usr/local/lib/python3.6/dist-packages/art/attacks/extraction/functionally_equivalent_extraction.py:232: RuntimeWarning: divide by zero encountered in true_divide
  t_hat = t_1 + np.divide(y_2 - y_1 - (t_2 - t_1) * m_2, m_1 - m_2)
/usr/local/lib/python3.6/dist-packages/art/attacks/extraction/functionally_equivalent_extraction.py:233: RuntimeWarning: divide by zero encountered in true_divide
  y_hat = y_1 + m_1 * np.divide(y_2 - y_1 - (t_2 - t_1) * m_2, m_1 - m_2)
/usr/local/lib/python3.6/dist-packages/art/attacks/extraction/functionally_equivalent_extraction.py:247: RuntimeWarning: invalid value encountered in less
  np.sum(np.abs((y_hat - y) / y) < rel_diff_value) > fraction_true * self.num_classes
/usr/local/lib/python3.6/dist-packages/art/attacks/extraction/functionally_equivalent_extraction.py:228: RuntimeWarning: divide by zero encountered in true_divide
  if np.sum(np.abs((m_1 - m_2) / m_1) < rel_diff_slope) > fraction_true * self.num_classes:
/usr/local/lib/python3.6/dist-packag

In [0]:
y_test_predicted_extracted = bbc.predict(x_test)
y_test_predicted_target = target_classifier.predict(x_test)

print("Target model - Test accuracy:", score_target[1])
print(
    "Extracted model - Test accuracy:",
    np.sum(np.argmax(y_test_predicted_extracted, axis=1) == np.argmax(y_test, axis=1)) / y_test.shape[0],
)
print(
    "Extracted model - Test Fidelity:",
    np.sum(np.argmax(y_test_predicted_extracted, axis=1) == np.argmax(y_test_predicted_target, axis=1))
    / y_test_predicted_target.shape[0],
)